In [137]:
import pandas as pd
from dotenv import load_dotenv
import os
import google.generativeai as genai
import json
import time
import sqlite3
load_dotenv()

True

Define fields except firm name

In [138]:
fields = ["Registered_Address", "CEO", "Establishment_Year", "Number_Of_Employees", "Revenue_Size" ,
        "Website", "NAICS_Code", "SIC_Code", "Status" ]

I'm using this to test how getting predictions just using gemini (without the databases) works

In [139]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="models/gemini-1.5-pro-002",
  # generation_config=generation_config,
)

# chat_session = model.start_chat(
#   history=[
#   ]
# )

Test responses

In [140]:
# response = chat_session.send_message("How many people died in the recent floods in Valencia?", tools='google_search_retrieval')
response = model.generate_content(contents="What is the land area of Spain?", tools='google_search_retrieval' )
print(response.text)

Spain's land area is approximately 500,000 square kilometers.  A more precise figure, according to the World Bank's 2021 data, is 499,733 square kilometers.  This excludes inland water bodies and Spain's exclusive economic zone. Including its island territories, the total area increases to around 505,000 square kilometers. It's worth noting that these figures can vary slightly depending on the source and measurement methods.  For example, Wikipedia cites the land area as 504,782 square kilometers within the Iberian Peninsula.

Spain is the largest country in Southern Europe, second largest in Western Europe (after France), and the fourth largest on the European continent (after Russia, Ukraine, and France).  Its extensive land area contributes to a wide variety of geographical features, ranging from mountainous regions and a large central plateau to coastal plains.  This diversity creates distinct regional landscapes and climates across the country.

It is important to note that this i

Function to form prompt to give gemini

Make sure gemini returns one answer or Null to each query instead of a paragraph

In [141]:
answer_format_v1 = """
If you have absolutely no idea on how to answer my query please write 'No information found'. 
Do not leave the answer blank or give me a long explanation on why you couldn't find the answer or an explanation of your answer.
If you find multiple possible answers return your best answer.
To reiterate i just want you to give a short answer in the format described above. Do not give me a long textual response."""

In [142]:
def form_prompt(query):
    prompt = f"""
    Context:
    {query}.
    {answer_format_v1}
    """

    return prompt

Define the queries we will give Gemini for each field. We need to specify the type of output we expect etc. May need some prompt engineering here

In [143]:
field_to_query_v1 = {
        "Registered_Address": "Tell me the Registered Adress for the firm named {firm_name}. Your answer should consist of the street address, city, state, country, and postal code.",
        "CEO": "Tell me the name and surname of the CEO for the firm named {firm_name}. Your answer should be in the format 'Name Surname'",
        "Establishment_Year": "Tell me the establishment year of {firm_name}. Your answer should be 4 digits in the format YYYY",
        "Number_Of_Employees": "Tell me the estimated number of employees for the firm named {firm_name}. Your answer should be an approximate range, like 1-10, 10-100, 100-1000, 1000-10000, 10000+ etc. You can specify other ranges like 200-250 if you wish.",
        "Revenue_Size": "Tell me the estimated annual revenue in dollars for the firm named {firm_name}. Your answer should be an approximate range, 10000-100000, 1000000-10000000, or 1000000000+ etc. You can specify other ranges like 200-250 if you wish.",
        "Website": "Tell me the url of the official website of for the firm named {firm_name}. If you can't find a website for the firm, please write 'No website found'",
        "NAICS_Code": "Tell me the numeric NAICS code for the firm named {firm_name}.",
        "SIC_Code":  "Tell me the numeric SIC code for the firm named {firm_name}.",
        "Status": "Tell me the whether the firm named {firm_name} is Active or Dissolved. Answer with one word.",
}

field_to_query = field_to_query_v1

TODO - Check if gemini remembers recent context like below. May be helpful to give a general context first, then ask each query, making sure the context resets with each firm

### Open firm databases

In [144]:
# Table to store the results
conn = sqlite3.connect("firm_database_gemini_with_grounding.db")
cursor = conn.cursor()


In [145]:
# Check existing tables and drop if you want

# Check existing tables
existing_tables = cursor.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
print(existing_tables)

[('sqlite_sequence',), ('firm_properties_gemini_with_grounding_v1',)]


Drop tables if needed

In [146]:
# table_to_drop = "firm_properties_gemini_with_grounding_v1"
# cursor.execute(f"DROP TABLE IF EXISTS {table_to_drop} ")
# conn.commit()

Create table

In [147]:
# You can create different tables for different prompt types, gemini model etc, if you rename the table
table_name = "firm_properties_gemini_with_grounding_v1"
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {table_name} (
               id INTEGER PRIMARY KEY,
               Firm_Name TEXT NOT NULL,
               Registered_Address TEXT,
               CEO TEXT,
               Establishment_Year INT,
               Number_Of_Employees TEXT,
               Revenue_Size TEXT,
               Website TEXT,
               NAICS_Code INT,
               SIC_Code INT,
               Status TEXT
               )
               ''')

Check how many rows are in the current table

In [148]:
cursor.execute(f"SELECT COUNT(*) FROM {table_name}").fetchall()

[(42,)]

In [149]:
cursor.execute(f"SELECT  * FROM {table_name}").fetchall()

[(1,
  '"TRACTION POWER SYSTEMS, INC."',
  'No information found\n',
  'No information found\n',
  'No information found\n',
  '10-100\n',
  '$10,000,000-$50,000,000\n\n\n',
  'secheron.com\n',
  335311,
  3612,
  'Active\n'),
 (2,
  '"A DAY TO REMEMBER", LLC',
  'No information found\n',
  'No information found\n',
  'No information found\n',
  '1-10\n\n\n',
  '$100,000-$1,000,000\n\n\n',
  'adtr.com\n',
  711310,
  711130,
  'No information found\n'),
 (3,
  '" MALCO INC. -A NEVADA CORP.',
  '5188 Paradise Rd, Las Vegas, NV, USA, 89119\n\n\n',
  'Tom Mallo\n\n\n',
  1989,
  '100-1000\n\n\n',
  '$10,000,000-$50,000,000\n\n\n',
  'budgetlasvegas.com\n\n\n',
  721110,
  3469,
  'Active\n'),
 (4,
  ".BISHOP'S GATE RESIDENTIAL MORTGAGE TRUST",
  '8th Floor 100 Bishopsgate, London, United Kingdom, EC2N 4AG\n',
  'No information found\n\n\n',
  2001,
  '10-100\n',
  '$100,000,000-$1,000,000,000\n\n\n',
  'No website found\n\n\n',
  522292,
  624120,
  'Active\n'),
 (5,
  '"J.CRESCI FARMS, L

In [150]:
field = 'Registered_Address'
firm_id = 26
cursor.execute(f"SELECT {field} FROM {table_name} WHERE id = ? AND {field} IS NOT NULL", (firm_id,))
print(cursor.fetchone())
if cursor.fetchone() is not None:
    print(f"Field '{field}' already has data for firm , skipping.")

('No information found.\n\n\n',)


### Get the list of firms from the web search database so as to be consistent

In [151]:
conn_web_results = sqlite3.connect("../firms_web_search_results.db")
cursor_websearch = conn_web_results.cursor()

Select Subset of firms for testing

In [152]:
cursor_websearch.execute(''' SELECT id, Firm_Name FROM firms_web_search_results ORDER BY id ASC''')
firm_web_search_results = cursor_websearch.fetchall()
firm_web_search_results = firm_web_search_results[:100]
firm_web_search_results

[(1, '"TRACTION POWER SYSTEMS, INC."'),
 (2, '"A DAY TO REMEMBER", LLC'),
 (3, '" MALCO INC. -A NEVADA CORP.'),
 (4, ".BISHOP'S GATE RESIDENTIAL MORTGAGE TRUST"),
 (5, '"J.CRESCI FARMS, L.L.C."'),
 (6, '1 MAN ARMY GOURMET FOODS LLC'),
 (7, '10-Code, LLC'),
 (8, "'Dat Hit Da Spot' LLC"),
 (9, '" LESLIE A. FEAST CONSTRUCTION COMPANY, INC."'),
 (10, '"R" ROCKY RANCH, INC.'),
 (11, '"FLEMING COUNTY KIWANIS CLUB INCORPORATED."'),
 (12, '$6 & $8 FASHIONS, INC.'),
 (13, "'JOSE ALFREDO PEREZ LOPEZ LLC"),
 (14, '"B & S GRAIN, LTD."'),
 (15, '"THE SUMMIT, L.L.C."'),
 (16, '"Foam It" Spray Foam Insulation, LLC'),
 (17, '"C.C.C. INC."'),
 (18, '(N)SQUARED ADVISORY, LLC'),
 (19, '#forabetternevada LLC'),
 (20, '#2 BLACKBOARD PROPERTIES, LLC'),
 (21, '"THE TRADES" GROUP, INC.'),
 (22, '008 PROJECT MANAGEMENT, LLC'),
 (23, '" C" CLUB INC.'),
 (24, '"SENN"SATIONAL CREATIONS, LLC'),
 (25, "'4' WHEELER'S HUNTING CLUB"),
 (26, '"ECHO LODGE NO. 103, OF THE INDEPENDENT ORDER OF ODD FELLOWS"'),
 (27, '1 By 

### Main Loop

Fill database using a single query for each field

In [153]:

for web_search_result in firm_web_search_results:
    firm_id = web_search_result[0]
    firm_name = web_search_result[1]

    # Check if firm already exists, insert row if it doesnt

    cursor.execute(f"SELECT id FROM {table_name} WHERE id = ? AND Firm_Name = ?", (firm_id, firm_name,))
    firm_row = cursor.fetchone()
    if firm_row is None:
        print("Inserting new firm:, ", firm_name)
        cursor.execute(f"INSERT INTO {table_name} (Firm_Name) VALUES (?)", (firm_name,))
        firm_id = cursor.lastrowid  
    else:
        # Get the existing firm's id
        print("Found row for firm, ", firm_name)
        firm_id = firm_row[0]
    

    # Fill in each field
    for field in fields:
        # Check if the field value in the target database is NULL
        cursor.execute(f"SELECT {field} FROM {table_name} WHERE id = ? AND Firm_Name = ? AND {field} IS NOT NULL", (firm_id,firm_name,))
        if cursor.fetchone() is not None:
            print(f"Field '{field}' already has data for firm '{firm_name}', skipping.")
            continue
      

        # Query Gemini
        # Update queries with firm name
        updated_queries = {key: value.format(firm_name=firm_name) for key, value in field_to_query.items()}
        # pass promot ot gemini
        prompt = form_prompt(updated_queries[field])

        # Send the prompt to model, try 3 times to get a response
        success = False
        response = None 
        failure_counter = 0
        # while not success and failure_counter < 3:
        try:
            response = model.generate_content(prompt,tools='google_search_retrieval').text
            # print("Success for ", firm_name, field)
            success = True
        except Exception as e:
            print("Exception occured: ", e)
            time.sleep(1) 
            failure_counter += 1
                
        print(f"Gemini response for {firm_name} - {field}: ", response)
        # Update database
        cursor.execute(f"UPDATE {table_name} SET {field} = ? WHERE Firm_Name = ? AND id = ?",
            (response, firm_name, firm_id))
        conn.commit()

Found row for firm,  "TRACTION POWER SYSTEMS, INC."
Field 'Registered_Address' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'CEO' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'Establishment_Year' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'Number_Of_Employees' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'Revenue_Size' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'Website' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'NAICS_Code' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'SIC_Code' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Field 'Status' already has data for firm '"TRACTION POWER SYSTEMS, INC."', skipping.
Found row for firm,  "A DAY TO REMEMBER", LLC
Field 'Registered_Address' already has data for firm '"A DAY TO REMEMBER", LLC', skipping.

In [54]:
table_name

'firm_properties_gemini_with_grounding_v2'

TODO:
- strip new line from gemini responses
- problem with it still returning paragraphs
- limit token output, 
    - limit token output when p value is low